#########################################################################################################################################

# Silver Standard Dataset Creation

#########################################################################################################################################

# 0. Preparation

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import numpy as np
import re
from sklearn.utils.class_weight import compute_class_weight
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
df_reviews =pd.read_pickle(r'C:\Users\Ong Hui Ling\Dropbox\PC\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\clean_data2.pkl')

In [3]:
df_reviews.columns

Index(['place_id', 'name', 'address', 'latitude', 'longitude', 'types',
       'googleMapsUri', 'priceRange', 'place_overall_rating',
       'userRatingCount', 'extract_date', 'author_name', 'user_review_rating',
       'text', 'author_id', 'uniqueID', 'timestamp', 'main_category',
       'sub_category', 'state', 'operating_hours_cleaned', 'Monday_hours',
       'Tuesday_hours', 'Wednesday_hours', 'Thursday_hours', 'Friday_hours',
       'Saturday_hours', 'Sunday_hours', 'clean_name'],
      dtype='object')

In [4]:
# Filter to restaurant category
df_restaurant = df_reviews[df_reviews['main_category'] == 'restaurant'].copy()

print(f"DATA OVERVIEW:")
print(f"Total restaurant reviews: {len(df_restaurant):,}")
print(f"Unique restaurants: {df_restaurant['place_id'].nunique():,}")
print(f"Unique reviewers: {df_restaurant['author_id'].nunique():,}")
print(f"Date range: {df_restaurant['timestamp'].min()} to {df_restaurant['timestamp'].max()}")

# Check state distribution
print(f"GEOGRAPHIC DISTRIBUTION:")
print(df_restaurant['state'].value_counts())

DATA OVERVIEW:
Total restaurant reviews: 19,330
Unique restaurants: 3,866
Unique reviewers: 14,590
Date range: 2016-03-16 09:49:09 to 2025-10-26 03:19:38
GEOGRAPHIC DISTRIBUTION:
state
Kuala Lumpur       2625
Selangor           2300
Perak              1905
Johor              1705
Penang             1475
Pahang             1440
Sabah              1310
Kedah              1265
Negeri Sembilan    1220
Melaka             1120
Sarawak            1005
Putrajaya           625
Terengganu          490
Kelantan            440
Perlis              290
Labuan              115
Name: count, dtype: int64


# 1. Weak Supervision using Star Rating

**Strategies**:

1-2 stars → Negative

4-5 stars → Positive  

3 stars → Neutral (EXCLUDED)

In [5]:
# Check rating distribution
print(f"RATING DISTRIBUTION:")
print(df_restaurant['user_review_rating'].value_counts().sort_index())

RATING DISTRIBUTION:
user_review_rating
1     1198
2      709
3     1996
4     4886
5    10541
Name: count, dtype: int64


**Observations:**
Class imbalance, more 4 and 5 stars reviews than negative reviews (1 and 2 stars)

In [6]:
# ============================================================
# ASSIGN WEAK LABEL
# ============================================================
def assign_weak_label(rating):
    """
    Assign weak sentiment labels based on star ratings
    Following Zhao et al. (2018) 
    
    1-2 stars → Negative
    4-5 stars → Positive  
    3 stars → Neutral (EXCLUDED)
    """
    if rating <= 2:
        return 'negative'
    elif rating >= 4:
        return 'positive'
    else:
        return 'neutral'  

In [7]:
# Apply to restaurant data
df_restaurant['weak_label'] = df_restaurant['user_review_rating'].apply(assign_weak_label)

print("="*70)
print("WEAK LABEL DISTRIBUTION (Before Filtering)")
print("="*70)
print(df_restaurant['weak_label'].value_counts())

WEAK LABEL DISTRIBUTION (Before Filtering)
weak_label
positive    15427
neutral      1996
negative     1907
Name: count, dtype: int64


In [8]:
# ============================================================
# FILTER OUT 3-STAR REVIEWS
# ============================================================
print("\n" + "="*70)
print("FILTERING OUT 3-STAR (NEUTRAL) REVIEWS")
print("="*70)

# Keep only positive (4-5 stars) and negative (1-2 stars)
df_filtered = df_restaurant[df_restaurant['weak_label'] != 'neutral'].copy()

print(f"\nBEFORE FILTERING:")
print(f"Total reviews: {len(df_restaurant):,}")
print(f"  - 5 stars: {(df_restaurant['user_review_rating']==5).sum():,}")
print(f"  - 4 stars: {(df_restaurant['user_review_rating']==4).sum():,}")
print(f"  - 3 stars: {(df_restaurant['user_review_rating']==3).sum():,} ← REMOVED")
print(f"  - 2 stars: {(df_restaurant['user_review_rating']==2).sum():,}")
print(f"  - 1 stars: {(df_restaurant['user_review_rating']==1).sum():,}")

print(f"\nAFTER FILTERING:")
print(f"Total reviews: {len(df_filtered):,}")
print(f"  - Positive (4-5 stars): {(df_filtered['weak_label']=='positive').sum():,}")
print(f"  - Negative (1-2 stars): {(df_filtered['weak_label']=='negative').sum():,}")

# Calculate class balance
positive_pct = (df_filtered['weak_label']=='positive').sum() / len(df_filtered) * 100
negative_pct = (df_filtered['weak_label']=='negative').sum() / len(df_filtered) * 100

print(f"\nCLASS BALANCE:")
print(f"Positive: {positive_pct:.1f}%")
print(f"Negative: {negative_pct:.1f}%")



FILTERING OUT 3-STAR (NEUTRAL) REVIEWS

BEFORE FILTERING:
Total reviews: 19,330
  - 5 stars: 10,541
  - 4 stars: 4,886
  - 3 stars: 1,996 ← REMOVED
  - 2 stars: 709
  - 1 stars: 1,198

AFTER FILTERING:
Total reviews: 17,334
  - Positive (4-5 stars): 15,427
  - Negative (1-2 stars): 1,907

CLASS BALANCE:
Positive: 89.0%
Negative: 11.0%


Observation:
- Severe class imbalance!!! (89.0% positive) 

Decision:
- Consider class weights in training

In [9]:
# Check state distribution after filtering
print(f"GEOGRAPHIC DISTRIBUTION (After Filtering):")
print(df_filtered['state'].value_counts().head(10))

GEOGRAPHIC DISTRIBUTION (After Filtering):
state
Kuala Lumpur       2405
Selangor           2066
Perak              1665
Johor              1542
Penang             1347
Pahang             1263
Sabah              1170
Kedah              1153
Negeri Sembilan    1071
Melaka             1015
Name: count, dtype: int64


# 2. Assign Class Weight in Handling Imbalance Data

Calculate the Weights using formula below: 

Weight = Total_Samples / (Number_of_Classes * Class_Count)

In [10]:
# 1. Manually input your counts 
# We know: 17,334 Total, 1,907 Negative (Label 0), 15,427 Positive (Label 1)
y_train = [0] * 1907 + [1] * 15427 

# 2. Compute Weights
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)

# 3. Print Results
print(f"Negative Weight (Class 0): {weights[0]:.4f}")
print(f"Positive Weight (Class 1): {weights[1]:.4f}")

Negative Weight (Class 0): 4.5448
Positive Weight (Class 1): 0.5618


# 3. Noise Reduction: Sentiment Consistency Protocol

In [11]:
# ============================================================
# CONSISTENCY FILTERING (Reduce Label Noise)
# ============================================================

# 1. Initialize VADER
vader = SentimentIntensityAnalyzer()

print("Calculating VADER scores...")

# 2. Create the 'vader_score' column FIRST
# We handle NaN/Empty values by assigning them a score of 0.0
df_filtered['vader_score'] = df_filtered['text'].apply(
    lambda x: vader.polarity_scores(str(x))['compound'] if pd.notna(x) and str(x).strip() != "" else 0.0
)

# 3. Define the Consistency Logic (Using the 0.5 Threshold)
def check_consistency_row(row):
    """
    Check if text sentiment matches weak label from rating
    Returns True if consistent, False if inconsistent
    
    Following Chen et al., (2025) and Sharifpour & Lahmiri, (2023)
    """
    score = row['vader_score']
    label = row['weak_label']
    
    if label == 'positive':
        # Keep unless STRONGLY negative (worse than -0.5)
        return score > -0.5
    elif label == 'negative':
        # Keep unless STRONGLY positive (better than 0.5)
        return score < 0.5
    else:
        return True

# 4. Apply the Logic
df_filtered['is_consistent'] = df_filtered.apply(check_consistency_row, axis=1)


Calculating VADER scores...


In [12]:
print(f"Reviews before filtering: {len(df_filtered):,}")
print(f"Consistent reviews: {df_filtered['is_consistent'].sum():,}")
print(f"Inconsistent reviews: {(~df_filtered['is_consistent']).sum():,}")
print(f"Inconsistency rate: {(~df_filtered['is_consistent']).sum()/len(df_filtered)*100:.1f}%")

# Compare with Zhao et al. (2018): 13.4% noise
print(f"\nComparison with literature:")
print(f"Zhao et al. (2018): 13.4% label noise")
print(f"This data: {(~df_filtered['is_consistent']).sum()/len(df_filtered)*100:.1f}% label noise")

# Keep only consistent reviews
df_silver_standard = df_filtered[df_filtered['is_consistent']].copy()

print(f"\nSILVER STANDARD DATASET:")
print(f"Total reviews: {len(df_silver_standard):,}")
print(f"Positive: {(df_silver_standard['weak_label']=='positive').sum():,}")
print(f"Negative: {(df_silver_standard['weak_label']=='negative').sum():,}")

Reviews before filtering: 17,334
Consistent reviews: 16,626
Inconsistent reviews: 708
Inconsistency rate: 4.1%

Comparison with literature:
Zhao et al. (2018): 13.4% label noise
This data: 4.1% label noise

SILVER STANDARD DATASET:
Total reviews: 16,626
Positive: 15,338
Negative: 1,288


# 4. Text Analysis

In [13]:
# Calculate text lengths
df_silver_standard['text_length'] = df_silver_standard['text'].str.len()
df_silver_standard['word_count'] = df_silver_standard['text'].str.split().str.len()

print("="*60)
print("TEXT CHARACTERISTICS ANALYSIS")
print("="*60)

print("\nTEXT LENGTH STATISTICS:")
print(df_silver_standard['text_length'].describe())

print("\nWORD COUNT STATISTICS:")
print(df_silver_standard['word_count'].describe())

TEXT CHARACTERISTICS ANALYSIS

TEXT LENGTH STATISTICS:
count    16626.000000
mean       391.442079
std        307.357374
min          6.000000
25%        197.000000
50%        308.000000
75%        488.000000
max       4093.000000
Name: text_length, dtype: float64

WORD COUNT STATISTICS:
count    16626.000000
mean        68.800974
std         53.626057
min          1.000000
25%         35.000000
50%         54.000000
75%         86.000000
max        684.000000
Name: word_count, dtype: float64


**Observation:**
- mean word count 68.8 -> It means the average Malaysian review in the dataset is ~69 words long.
- max Length -> The longest review is 684 words. That is a huge essay.

In [14]:
# Check for very short/long reviews
print(f"\nDATA QUALITY CHECKS:")
print(f"Reviews < 5 words: {(df_silver_standard['word_count'] < 5).sum():,}")
print(f"Reviews > 200 words: {(df_silver_standard['word_count'] > 200).sum():,}")
print(f"Empty reviews: {df_silver_standard['text'].isna().sum():,}")


DATA QUALITY CHECKS:
Reviews < 5 words: 16
Reviews > 200 words: 495
Empty reviews: 0


In [15]:
# Sample reviews
print(f"\nSAMPLE POSITIVE REVIEWS:")
print(df_silver_standard[df_silver_standard['weak_label']=='positive']['text'].head(3).tolist())

print(f"\nSAMPLE NEGATIVE REVIEWS:")
print(df_silver_standard[df_silver_standard['weak_label']=='negative']['text'].head(3).tolist())


SAMPLE POSITIVE REVIEWS:
['Aaliya Famansara serves some of the best crab curry and prawn curry around… rich, aromatic, and full of authentic spice flavours. The seafood is fresh and cooked perfectly. Don’t miss the sweet appam with brown sugar and coconut cream - the perfect finish to the meal. A must-visit for true Malaysian comfort food!', "The food, service and atmosphere is top class and worthy of a fine dining establishment. Everything I had was good quality and tasted fine, although some dishes worked better than others.\n\nThe meal started on a high note with some complimentary Rasam served in small cups. I'm not sure what type of rasam it was but it may possibly be the tastiest rasam I've ever had. Compliments to the Chef!\n\nThe Fish Cutlets were tasty but had a strong fishy aroma, perhaps from the type of fish used? The Lamb Cutlets were better in terms of taste and aroma and I preferred it over the fish.\n\nThe stars of the meal for me were the Sambols or salads. The Gottu 

In [16]:
# Check for Manglish patterns
manglish_patterns = ['sedap', 'lambat', 'giler', 'laju', 'sangat', 'sikit', 
                     'banyak', 'panas', 'sejuk', 'makan', 'tapau', 'bungkus', 
                     'mamak', 'kureng', 'masin', 'wok-hei', 'manis', 'pedas', 
                     'kick', 'shiok', 'sambal', 'goreng', 'kantoi', 'boss',
                     'amoi', 'anneh',  'abang', 'kakak', 'auntie', 'uncle', 
                     'mahal', 'murah', 'berbaloi', 'kecil', 'besar', 'jauh', 
                     'dekat', 'senyap', 'sesak', 'solat', 'bising', 'senyap',
                     'nasi', 'roti',  'kuah', 'nasi lemak', 'char kuey teow', 
                     'roti canai', 'laksa', 'satay', 'rendang', 'dim sum', 
                     'wan tan mee', 'hokkien mee', 'chicken rice', 'nasi kandar', 
                     'banana leaf', 'tomyam', 'cendol', 'teh tarik', 'kopi', 
                     'kek', 'kerabu', 'bak kut teh', 'yong tau foo', 'naan', 
                     'otak-otak', 'kuih', 'apam', 'chee cheong fun', 'biryani',
                     'ikan', 'ayam']
df_silver_standard['has_manglish'] = df_silver_standard['text'].str.lower().str.contains('|'.join(manglish_patterns), na=False)

print(f"\n🇲🇾 MANGLISH PREVALENCE:")
print(f"Reviews with Manglish terms: {df_silver_standard['has_manglish'].sum():,} ({df_silver_standard['has_manglish'].sum()/len(df_silver_standard)*100:.1f}%)")


🇲🇾 MANGLISH PREVALENCE:
Reviews with Manglish terms: 4,765 (28.7%)


In [17]:
df_silver_standard.to_csv(r'C:\Users\Ong Hui Ling\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\silver_std.csv')
df_silver_standard.to_pickle(r'C:\Users\Ong Hui Ling\Documents\Github\Aspect-Based-Sentiment-Analysis\Dataset\silver_std.pkl')